In [1]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override

#from models import VisualEncoder
from train import *
from wrappers_2 import *

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 512, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

In [3]:
class MyModelClass(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 512
        self.encoder = VisualEncoder()
        self.encoder.load_state_dict(
            torch.load("/IGLU-Minecraft/models/AnnaCNN/encoder_weigths.pth", map_location=torch.device('cpu'))
        )
        self.action_head = nn.Linear(features_dim, action_space.n)
        self.value_head = nn.Linear(features_dim, 1)
        self.last_value = None
        
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.encoder.cuda()
            self.action_head.cuda()
            self.value_head.cuda()
        
    @override(TorchModelV2)
    def forward(self, input_dict, state, seq_lens):
        obs = input_dict['obs'].permute(0, 3, 1, 2).float() / 255.0
        if self.use_cuda:
            obs.cuda()
            
        features = self.encoder(obs)
        action = self.action_head(features)
        self.last_value = self.value_head(features).squeeze(1)
        return action, state
    
    @override(TorchModelV2)
    def value_function(self):
        assert self.last_value is not None, "must call forward() first"
        return self.last_value

In [4]:
ModelCatalog.register_custom_model("my_torch_model", MyModelClass)

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, rew):
        if rew == 0:
            rew = -0.1
        return rew
    
def env_creator(env_config):
    env = gym.make('IGLUSilentBuilder-v0', max_steps=1000)
    env.update_taskset(TaskSet(preset=['C8']))
    env = PovOnlyWrapper(env)
    env = SelectAndPlace(env)
    env = Discretization(env, flat_action_space('human-level'))
    env = RewardWrapper(env)
    return env

from ray.tune.registry import register_env
register_env("my_env", env_creator)

from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

In [ ]:
from ray.tune.integration.wandb import WandbLogger

analysis = tune.run(PPOTrainer, 
         config={
             "env": "my_env", 
             "framework": "torch",
             "num_gpus": 1,
             "num_workers": 1,
             "sgd_minibatch_size": 256,
             "clip_param": 0.2,
             "entropy_coeff": 0.01,
             "lambda": 0.95,
             "train_batch_size": 1000,
             "model": {
                    # Specify our custom model from above.
                    "custom_model": "my_torch_model",
                    # Extra kwargs to be passed to your model's c'tor.
                    "custom_model_config": {},
              },
             "logger_config": {
                  "wandb": {
                      "project": "IGLU-Minecraft",
                      "name": "PPO C8 pretrained (AnnaCNN) (3 noops after placement and reward shaping)"
                  }
              }

        },
        loggers=[WandbLogger])

/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2021-10-09 21:38:54,838	INFO wandb.py:170 -- Already logged into W&B.
2021-10-09 21:38:54,853	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


Trial name,status,loc
PPO_my_env_4d34e_00000,RUNNING,


wandb: Currently logged in as: linar (use `wandb login --relogin` to force relogin)
wandb: WARNING Tried to auto resume run with id 5d62f_00000 but id 4d34e_00000 is set.
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
(pid=258828) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=258828)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=258828) 2021-10-09 21:38:58,243	INFO ppo.py:159 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=258828) 2021-10-09 21:38:58,243	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=258822) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=258822)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=258828) 2021-10-09 21:39:03,962	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(pid=258822) /root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=258822)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=258822) /root/miniconda/envs/py37/lib/py

Result for PPO_my_env_4d34e_00000:
  agent_timesteps_total: 1000
  custom_metrics: {}
  date: 2021-10-09_21-40-05
  done: false
  episode_len_mean: 484.0
  episode_media: {}
  episode_reward_max: -45.20000000000037
  episode_reward_mean: -48.40000000000042
  episode_reward_min: -51.60000000000046
  episodes_this_iter: 2
  episodes_total: 2
  experiment_id: 1667dda7b06542909a1096e12654b23d
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.19999999999999996
          cur_lr: 5.000000000000001e-05
          entropy: 1.6374062498410542
          entropy_coeff: 0.009999999999999998
          kl: 0.011904515929222962
          policy_loss: -0.0018852401110861037
          total_loss: 0.21260175696677633
          vf_explained_var: 0.5787085890769958
          vf_loss: 0.2284801604019271
    num_agent_steps_sampled: 1000
    num_agent_steps_trained: 1000
    num_steps_sampled: 1000
    num_steps_trained: 1000
  iteratio

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_4d34e_00000,RUNNING,192.168.3.5:258828,85,1746.92,85000,-43.332,-29.3,-54.1,433.55


Result for PPO_my_env_4d34e_00000:
  agent_timesteps_total: 86000
  custom_metrics: {}
  date: 2021-10-09_22-08-31
  done: false
  episode_len_mean: 435.48
  episode_media: {}
  episode_reward_max: -30.700000000000166
  episode_reward_mean: -43.525000000000354
  episode_reward_min: -54.1000000000005
  episodes_this_iter: 2
  episodes_total: 199
  experiment_id: 1667dda7b06542909a1096e12654b23d
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.14238281249999996
          cur_lr: 5.000000000000001e-05
          entropy: 1.879926500055525
          entropy_coeff: 0.009999999999999998
          kl: 0.009627042105578711
          policy_loss: 0.054338582936260434
          total_loss: 0.6161935482588079
          vf_explained_var: 0.2281894087791443
          vf_loss: 0.5792835054712163
    num_agent_steps_sampled: 86000
    num_agent_steps_trained: 86000
    num_steps_sampled: 86000
    num_steps_trained: 86000
  ite

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_4d34e_00000,RUNNING,192.168.3.5:258828,86,1765.84,86000,-43.525,-30.7,-54.1,435.48


Result for PPO_my_env_4d34e_00000:
  agent_timesteps_total: 87000
  custom_metrics: {}
  date: 2021-10-09_22-08-50
  done: false
  episode_len_mean: 437.85
  episode_media: {}
  episode_reward_max: -32.100000000000186
  episode_reward_mean: -43.76200000000036
  episode_reward_min: -54.1000000000005
  episodes_this_iter: 2
  episodes_total: 201
  experiment_id: 1667dda7b06542909a1096e12654b23d
  hostname: linar-Z390-GAMING-X
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.14238281249999996
          cur_lr: 5.000000000000001e-05
          entropy: 1.8463498089048598
          entropy_coeff: 0.009999999999999998
          kl: 0.012046026045272745
          policy_loss: -0.09771091938018799
          total_loss: 1.0890718814399507
          vf_explained_var: 0.3711315095424652
          vf_loss: 1.2035311463806364
    num_agent_steps_sampled: 87000
    num_agent_steps_trained: 87000
    num_steps_sampled: 87000
    num_steps_trained: 87000
  ite

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_my_env_4d34e_00000,RUNNING,192.168.3.5:258828,87,1784.7,87000,-43.762,-32.1,-54.1,437.85
